In [1]:
from Binance import Binance
import datetime
import pandas as pd
import time
from pandas import json_normalize
import random

In [2]:

# todo: obtain data from Binance
def kline_data(stime, etime, symbol = 'BTCUSDT', interval = '1d'):
    
    # stimestamp = int(datetime.datetime.strptime(stime, "%Y-%m-%d").timestamp()*1000)
    # etimestamp = int(datetime.datetime.strptime(etime, "%Y-%m-%d").timestamp()*1000)
    params = {
        'symbol': symbol,
        'interval': interval,
        'startTime': stime,
        'endTime': etime,
        'limit': 1000,
    }
    # url_path = '/fapi/v1/klines'
    response = api.send_public_request(url_path=url_path,payload = params)
    data = pd.DataFrame(response)

    col = ['time','Open','High','Low','Close','Volume','Close time','Quote asset volume','Number of trades',
        'Taker buy volume','Taker buy quote asset volume','Ignore']

    data.columns = col
    #       transfer the timestamp into time
    data['time'] = pd.to_datetime(data['time'],unit='ms',utc=True)
    data['Close time'] = pd.to_datetime(data['Close time'],unit='ms',utc=True)
    
    data.iloc[:,[1,2,3,4,5,7,9,10]] = data.iloc[:,[1,2,3,4,5,7,9,10]].astype(float)
    
    return data


def time_format(TimeString):
    TimeString = datetime.datetime.strptime(TimeString, "%Y-%m-%d %H:%M:%S")
    TimeString = datetime.datetime.timestamp(TimeString)*1000
    
    return str(int(TimeString))

def start_end_time(startTime,interval,length = 1000):
#     Due to the limit of the api, we need a function to help cut a long time into small period.
#     The function input the start time and interval and return the start time and endtime. 
    
    startTime = int(startTime)/1000
    startTime = datetime.datetime.fromtimestamp(startTime)
    
#     unit 
    unit = interval
#     transfer the interval frequence
    if unit in ["1m","3m","5m","15m","30m"]:
        unit = 'm'
    elif unit in ["1h","2h","4h","6h","8h","12h"]:
        unit = 'hours'
    elif unit in ["1d","3d"]:
        unit = 'D'
    else:
        unit = 'W'
        
    endTime = pd.to_datetime(startTime)+pd.to_timedelta(length, unit = unit)

    startTime = datetime.datetime.timestamp(startTime)*1000
    endTime = datetime.datetime.timestamp(endTime)*1000
    
    return str(int(startTime)),str(int(endTime))


def Long_data(pair,startTime,endTime,interval):
    data = pd.DataFrame()

    while int(endTime) > int(startTime):
        try :
            [startTime,eTime] = start_end_time(startTime=startTime,interval=interval)
            
            if int(eTime) >=int(endTime):
                eTime = endTime
                
            d = kline_data(stime = startTime, etime = eTime, symbol = pair, interval = interval)
            
            # log for the time
            # print(datetime.datetime.fromtimestamp(int(startTime)/1000),'finished!')
            startTime = eTime
            data = data.append(d)
        except :
            startTime = eTime
            time.sleep(0.1)
            continue
        time.sleep(0.1)
    return data

In [3]:
api_key = 'your_api_key'
api_secret = 'your_api_secret'
BASE_URL = 'https://api.binance.com'


api = Binance(BASE_URL = BASE_URL, api_key=api_key, api_secret=api_secret)
# print(result)
# symbol = 'BTCUSDT'
# interval = '1d'
# startTime = '2017-12-30 00:00:00'
# startTime = time_format(startTime)

# endTime = '2022-06-20 00:00:00'
# endTime = time_format(endTime)
# result = Long_data(symbol,startTime,endTime,interval)

In [4]:
url_path = '/api/v3/exchangeInfo'
params = {
}
response = api.send_public_request(url_path, params)
exchange = json_normalize(response['symbols'])
symbols_ls = exchange.symbol

In [7]:
url_path = '/api/v3/klines'
symbol = 'BTCUSDT'
interval = '1d'
startTime = '2015-12-30 00:00:00'
startTime = time_format(startTime)

endTime = '2022-06-26 00:00:00'
endTime = time_format(endTime)

def token_pct(symbol):
    data = Long_data(symbol,startTime,endTime,interval)
    data[symbol] = (data.Close/data.Open - 1)*100
    data.time = data.time.dt.strftime('%Y-%m-%d')
    data = data[['time',symbol]]
    return data

def token_vol(smb):
    data = Long_data(smb,startTime,endTime,interval)
    data.time = data.time.dt.strftime('%Y-%m-%d')
    data = data[['time','Quote asset volume']]
    data = data.rename(columns={"Quote asset volume": smb})
    return data

# All token together

In [37]:
all_data = []
error_coin = []
for i in range(len(symbols_ls)):
    time.sleep(random.randint(1,9)*0.1)
    try: 
        dt = token_pct(symbols_ls[i])
        print(i)
        all_data.append(dt)
    except:
        print(symbols_ls[i])
        error_coin.append(symbols_ls[i])
        
import functools

final_dt = functools.reduce(lambda x, y: x.merge(y,how = 'outer', on = 'Time'), all_data)
final_dt.to_csv('AllCoinPct_2016tonow.csv')

0
1
2
3
4
5
6
7
8
9
10
BTCUSDT
12
13
14
15
16


# Multiple quoting assets

In [11]:
quote_asset = exchange.quoteAsset.value_counts().index[:5]

In [12]:
btc_quoteasset_symbol = exchange[exchange.quoteAsset == quote_asset[0]].symbol
usdt_quoteasset_symbol = exchange[exchange.quoteAsset == quote_asset[1]].symbol
busd_quoteasset_symbol = exchange[exchange.quoteAsset == quote_asset[2]].symbol
bnb_quoteasset_symbol = exchange[exchange.quoteAsset == quote_asset[3]].symbol
eth_quoteasset_symbol = exchange[exchange.quoteAsset == quote_asset[4]].symbol
others_quoteasset_symbol = exchange[~exchange.quoteAsset.isin(quote_asset)].symbol

In [13]:
import functools
def category_pct(symbols_ls, name):
    print(symbols_ls[-5:])
    symbols_ls = symbols_ls.reset_index(drop=True)
    all_data = []
    error_coin = []
    for i in range(len(symbols_ls)):
        time.sleep(random.randint(1,9)*0.1)
        try: 
            dt = token_vol(symbols_ls[i])
            print(i)
            all_data.append(dt)
        except:
            print(symbols_ls[i])
            error_coin.append(symbols_ls[i])
            
    final_dt = functools.reduce(lambda x, y: x.merge(y,how = 'outer', on = 'time'), all_data)
    final_dt.to_csv(f'{name}_2016tonow.csv')

In [14]:
# category_pct(btc_quoteasset_symbol, 'btc')
# category_pct(usdt_quoteasset_symbol, 'usdt')
category_pct(busd_quoteasset_symbol, 'busd')
category_pct(bnb_quoteasset_symbol, 'bnb')
category_pct(eth_quoteasset_symbol, 'eth')
category_pct(others_quoteasset_symbol, 'others')

2063     ASRBUSD
2064    FIROBUSD
2065     NKNBUSD
2068     GTOBUSD
2070    WBTCBUSD
Name: symbol, dtype: object
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
2